In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount("/content/Drive/")

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive
import textwrap
import progressbar
import keras
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import time
import datetime
import json

In [ ]:
f_train_CL = open("/content/Drive/MyDrive/Technical/RR/CL_train.json", "r")
f_test_CL = open("/content/Drive/MyDrive/Technical/RR/CL_test.json", "r")
f_test_IT = open("/content/Drive/MyDrive/Technical/RR/IT_test.json", "r")
f_train_IT = open("/content/Drive/MyDrive/Technical/RR/IT_train.json", "r")

data_tr_CL = json.load(f_train_CL)
f_train_CL.close()
data_te_CL = json.load(f_test_CL)
f_test_CL.close()
data_tr_IT = json.load(f_train_IT)
f_train_IT.close()
data_te_IT = json.load(f_test_IT)
f_test_IT.close()

Make dev jason files

In [ ]:
def df_maker(data, d_t):
  docs = data.keys()
  texts = []
  labels = []
  label_mapper = {}
  if(d_t == "compressed"):
    label_mapper = {
        "Fact": 0,
        "ArgumentPetitioner": 1,
        "ArgumentRespondent": 1,
        "RulingByPresentCourt": 2,
        "RulingByLowerCourt": 3,
        "RatioOfTheDecision": 4,
        "None": -1,
        "Statute": 5,
        "PrecedentReliedUpon": 6,
        "PrecedentNotReliedUpon": 6,
        "PrecedentOverruled": 6,
        "Issue": 0,
        "Dissent": 0
    }
  if(d_t == "full"):
    label_mapper = {
        "Fact": 0,
        "ArgumentPetitioner": 1,
        "ArgumentRespondent": 2,
        "RulingByPresentCourt": 3,
        "RulingByLowerCourt": 4,
        "RatioOfTheDecision": 5,
        "None": 6,
        "Statute": 7,
        "PrecedentReliedUpon": 8,
        "PrecedentNotReliedUpon": 9,
        "PrecedentOverruled": 10,
        "Issue": 11,
        "Dissent": 12
    }
  for doc in docs:
    for i,label in enumerate(data[doc]["complete"]):
      if(label == "None" and d_t=="compressed"):
        continue

      texts.append(data[doc]["sentences"][i])
      labels.append(label_mapper[label])

  dict_tl = {"text": texts, "label": labels}
  df = pd. DataFrame(dict_tl)
  return df

In [ ]:
d_type_start = "compressed"
train_CL = df_maker(data_tr_CL, d_type_start)
test_CL = df_maker(data_te_CL, d_type_start)
train_IT = df_maker(data_tr_IT, d_type_start)
test_IT = df_maker(data_te_IT, d_type_start)

CL_train_set = train_CL
CL_validation_set = train_CL
CL_test_set = test_CL
IT_train_set = train_IT
IT_validation_set = train_IT
IT_test_set = test_IT

In [ ]:
from transformers import PreTrainedModel, PreTrainedTokenizer, PretrainedConfig
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig
from transformers import XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig
from transformers import XLMForSequenceClassification, XLMTokenizer, XLMConfig
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig

MODEL_CLASSES = {
    'bert': (BertForSequenceClassification, BertTokenizer, BertConfig),
    'xlnet': (XLNetForSequenceClassification, XLNetTokenizer, XLNetConfig),
    'xlm': (XLMForSequenceClassification, XLMTokenizer, XLMConfig),
    'roberta': (RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig),
    'distilbert': (DistilBertForSequenceClassification, DistilBertTokenizer, DistilBertConfig)}

model_type = 'bert' ###--> CHANGE WHAT MODEL YOU WANT HERE!!! <--###
model_class, tokenizer_class, config_class = MODEL_CLASSES[model_type]
model_name = 'bert-base-uncased'

In [ ]:
def att_masking(input_ids):
  attention_masks = []
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)
  return attention_masks

In [ ]:
def input_id_maker(dataf, tokenizer):
  input_ids = []
  lengths = []
  for i in progressbar.progressbar(range(len(dataf['text']))):
    sen = dataf['text'].iloc[i]
    sen = tokenizer.tokenize(sen)
    #taking the last 510 tokens
    #you can try out multiple combinations of input tokens as we did in the paper
    if(len(sen) > 254):
      sen = sen[len(sen)-254:]

    encoded_sent = tokenizer.encode(sen, add_special_tokens=True)
    input_ids.append(encoded_sent)
    lengths.append(len(encoded_sent))

  input_ids = pad_sequences(input_ids, maxlen=256, value=0, dtype="long", truncating="post", padding="post")
  return input_ids, lengths

In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
CL_train_input_ids, _ = input_id_maker(CL_train_set, tokenizer)
CL_validation_input_ids = CL_train_input_ids

In [ ]:
IT_train_input_ids, _ = input_id_maker(IT_train_set, tokenizer)
IT_validation_input_ids = IT_train_input_ids

In [ ]:
CL_train_attention_masks = att_masking(CL_train_input_ids)
CL_validation_attention_masks = CL_train_attention_masks

CL_train_labels = CL_train_set['label'].to_numpy().astype('int')
CL_validation_labels = CL_train_labels

In [ ]:
IT_train_attention_masks = att_masking(IT_train_input_ids)
IT_validation_attention_masks = IT_train_attention_masks

IT_train_labels = IT_train_set['label'].to_numpy().astype('int')
IT_validation_labels = IT_train_labels

In [ ]:
CL_test_input_ids, _ = input_id_maker(CL_test_set, tokenizer)
CL_test_attention_masks = att_masking(CL_test_input_ids)
CL_test_labels = CL_test_set['label'].to_numpy().astype('int')

In [ ]:
IT_test_input_ids, _ = input_id_maker(IT_test_set, tokenizer)
IT_test_attention_masks = att_masking(IT_test_input_ids)
IT_test_labels = IT_test_set['label'].to_numpy().astype('int')

#CL


In [ ]:
CL_train_inputs = CL_train_input_ids
CL_validation_inputs = CL_validation_input_ids
CL_train_masks = CL_train_attention_masks
CL_validation_masks = CL_validation_attention_masks

# IT_train_inputs = IT_train_input_ids
# IT_validation_inputs = IT_validation_input_ids
# IT_train_masks = IT_train_attention_masks
# IT_validation_masks = IT_validation_attention_masks

train_inputs = torch.tensor(CL_train_inputs)
train_labels = torch.tensor(CL_train_labels)
train_masks = torch.tensor(CL_train_masks)
validation_inputs = torch.tensor(CL_validation_inputs)
validation_labels = torch.tensor(CL_validation_labels)
validation_masks = torch.tensor(CL_validation_masks)

# train_inputs = torch.tensor(IT_train_inputs)
# train_labels = torch.tensor(IT_train_labels)
# train_masks = torch.tensor(IT_train_masks)
# validation_inputs = torch.tensor(IT_validation_inputs)
# validation_labels = torch.tensor(IT_validation_labels)
# validation_masks = torch.tensor(IT_validation_masks)

In [ ]:
batch_size = 16
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size = batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size = batch_size)

In [ ]:
nl = 0
if(d_type_start == "full"):
  nl = 13
else:
  nl = 7
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=nl)
model.to(device)

In [ ]:
lr = 2e-5
max_grad_norm = 1.0
epochs = 3
num_total_steps = len(train_dataloader)*epochs
num_warmup_steps = 1000
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=True)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps, num_training_steps = num_total_steps)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

seed_val = 34
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}. '.format(step, len(train_dataloader)))

        
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)            
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
        
    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():        
          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy

        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

print("")
print("Training complete!")

In [ ]:
prediction_inputs = torch.tensor(CL_test_input_ids)
prediction_masks = torch.tensor(CL_test_attention_masks)
prediction_labels = torch.tensor(CL_test_labels)

# prediction_inputs = torch.tensor(IT_test_input_ids)
# prediction_masks = torch.tensor(IT_test_attention_masks)
# prediction_labels = torch.tensor(IT_test_labels)

prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []


# Predict 
for (step, batch) in enumerate(prediction_dataloader):
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  

  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

In [ ]:
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)
pred_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_labels.flatten()

flat_accuracy(predictions,true_labels)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
# utility function to calculate metric scores
def metrics_calculator(preds, test_labels):
    cm = confusion_matrix(test_labels, preds)
    TP = []
    FP = []
    FN = []
    for i in range(0,nl):
        summ = 0
        for j in range(0,nl):
            if(i!=j):
                summ=summ+cm[i][j]

        FN.append(summ)
    for i in range(0,nl):
        summ = 0
        for j in range(0,nl):
            if(i!=j):
                summ=summ+cm[j][i]

        FP.append(summ)
    for i in range(0,nl):
        TP.append(cm[i][i])
    precision = []
    recall = []
    for i in range(0,nl):
        precision.append(TP[i]/(TP[i] + FP[i]))
        recall.append(TP[i]/(TP[i] + FN[i]))

    macro_precision = sum(precision)/nl
    macro_recall = sum(recall)/nl
    micro_precision = sum(TP)/(sum(TP) + sum(FP))
    micro_recall = sum(TP)/(sum(TP) + sum(FN))
    micro_f1 = (2*micro_precision*micro_recall)/(micro_precision + micro_recall)
    macro_f1 = (2*macro_precision*macro_recall)/(macro_precision + macro_recall)
    return macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1

macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1 = metrics_calculator(pred_flat, labels_flat)
print(macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labels_flat, pred_flat))